In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth

In [21]:
EPSILON = 1 # epsilon sets the length of boxes, default 1
dim = 9 # dim is the number of embeddings' dimension, default 9

In [8]:
rawdata = np.load('sample_data/english_newlit_SVD_dict.npy', allow_pickle=True)[()]
# choose any other dictionary of a language

In [23]:
rawdata['cat']

array([ 0.01679443,  0.00414149, -0.01409523,  0.00645458, -0.00607685,
        0.0022039 , -0.00080252, -0.01132439,  0.04173432,  0.02197117,
        0.00837969,  0.00269821,  0.0148389 ,  0.00303758, -0.00176646,
       -0.0859572 ])

In [22]:
rawdata['cat'][-dim:]

array([-0.01132439,  0.04173432,  0.02197117,  0.00837969,  0.00269821,
        0.0148389 ,  0.00303758, -0.00176646, -0.0859572 ])

In [19]:
len(rawdata)

96582

In [53]:
datapoints = torch.zeros([len(rawdata), dim])

In [54]:
for i, (k, v) in enumerate(rawdata.items()):
  datapoints[i] = torch.tensor(v[-dim:])

In [58]:
datapoints[0]

tensor([ 1.7509e-04,  1.5773e-04, -7.5267e-05,  2.4380e-04,  1.4749e-04,
         4.4278e-04,  4.2872e-04, -6.8720e-05, -5.3378e-04])

In [ ]:
# the mins and maxes are needed to create 'a boundary of the language'

In [59]:
mins = torch.tensor([10000 for i in range(dim)])
for i in range(len(datapoints)):
  mins = np.minimum(mins, datapoints[i])
mins

tensor([ -4.2081,  -2.2523,  -3.9531,  -3.2601,  -2.8331,  -4.1316,  -2.8022,
        -11.7184, -61.3082], dtype=torch.float64)

In [60]:
maxs = torch.tensor([-10000 for i in range(dim)])
for i in range(len(datapoints)):
  maxs = np.maximum(maxs, datapoints[i])
maxs

tensor([ 1.3984e+00,  2.1375e+00,  4.1896e+00,  4.2405e+00,  4.7449e+00,
         3.3393e+00,  1.3635e+01,  9.4065e+00, -4.4903e-06],
       dtype=torch.float64)

In [61]:
[int(np.ceil((maxs[i] - mins[i])/1)) for i in range(dim)]

[6, 5, 9, 8, 8, 8, 17, 22, 62]

In [62]:
def box_counting(dims=9, dataset=torch.zeros(1,dim), epsilon=1):
  boxes = dict()
  for el in dataset:
    mod_coords = str(torch.Tensor.int(np.floor((el)/epsilon)).tolist())
    if boxes.get(mod_coords):
      boxes[mod_coords] += 1
    else:
      boxes[mod_coords] = 1
  return(boxes)

In [65]:
final_boxes = box_counting(dim, mins, maxs, datapoints, EPSILON)

In [75]:
point = str(torch.Tensor.int(np.floor((datapoints[0])/EPSILON)).tolist())
final_boxes[point]
# example: how many points are in the same box as the first word in the dataset

1929